In [2]:
!pip3 install python-levenshtein

     |████████████████████████████████| 51kB 3.7MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144803 sha256=40011036aa11a31f438a09489c5923c798acdaea00a06eb8dd637deb364478e3
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pdb
import string
import pandas as pd
from Levenshtein import distance
import scipy
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import model_from_json
from keras.utils import to_categorical

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X), Fs

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X), Fs

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

# if __name__=="__main__":
#     wavs = '/content/gdrive/MyDrive/EE698V/shared_train/audio_train/180126-4-4-2.wav'
#     x, Fs= wav2feat(wavs)
#     print(np.shape(x))
#     db = librosa.amplitude_to_db(x,ref=np.max)
#     a = librosa.display.specshow(db, sr=Fs, y_axis='log', x_axis='time')

#     # wavfiles = ['../shared_train/audio_train/180937-7-3-27.wav','../shared_train/audio_train/180937-7-3-27.wav']
#     # X = wavs2feat(wavfiles)
#     # eval('test_task1/labels.csv', 'test_task1/est.csv', 1)
#     editDistance("dog_bark-street_music-engine_idling",
#         "siren-street_music-engine_idling")

In [ ]:
# Remove exact same audio samples from dataset
list_of_files = os.listdir('/content/gdrive/MyDrive/EE698V/shared_train/audio_train_1ch/')
final_list = []
seen = set()
for i in list_of_files:
  if i.split('-')[0] in seen:
    continue:
  else:
    seen.add(i.split('-')[0])
    final_list.append(i)

final_list = np.array(final_list)
with open('/content/gdrive/MyDrive/EE698V/train_label/unique_files','wb') as f:
  np.save(f,final_list)

In [ ]:
# Load unique audio samples
with open('/content/gdrive/MyDrive/EE698V/train_label/unique_files','rb') as f:
  files_list = pickle.load(f)

In [ ]:
# Get spectrogam of audios and create training array of them
train_images = []
train_labels = np.array([])
file_path = '/content/gdrive/MyDrive/EE698V/shared_train/audio_train_1ch/'
i=1
for wavs in files_list:
  x, Fs= wav2feat(file_path + wavs)
  labl = int(wavs.split('-')[1])
  if i%100==0:
    print(i)
  i+=1
  train_images.append(np.array(x))
  train_labels = np.append(train_labels,labl)

100
200


In [ ]:
# Break each audio spectrogram into 40 frames each and put in a new array 
train_images2 = []
train_labels2 = []
for labl,i in enumerate(train_images):
  iter = int(i.shape[1]/40)
  for j in range(iter):
    train_images2.append(i[:,40*(j):40*(j+1)])
    train_labels2.append(train_labels[labl])

In [ ]:
# Reshape the array and one hot encode the labels for sending to network
train_images = np.array(train_images2)
train_images = train_images.reshape(train_images.shape+(1,))

train_labels = np.array(train_labels2)
train_labels = train_labels.astype(int)
train_labels = to_categorical(train_labels)

In [ ]:
# Re confirm the shapes before starting the training
print(train_images.shape)
print(train_labels.shape)

(1984, 10)

In [ ]:
# Validation data taken from other sources
if __name__=="__main__":
  testu_images = []
  testu_labels = []
  file_path = '/content/gdrive/MyDrive/EE698V/Test_Validation/'
  i = 1
  for wavs in os.listdir(file_path):
    if(i==300):
      break
    x, Fs= wav2feat(file_path + wavs)
    labl = int(wavs.split('-')[1])
    if i%100==0:
      print(i)
    i+=1
    testu_images.append(np.array(x))
    testu_labels.append(labl)

100
200


In [ ]:
testu_images2 = []
testu_labels2 = []
for labl,i in enumerate(testu_images):
  iter = int(i.shape[1]/40)
  for j in range(iter):
    if(j==10):
      break
    testu_images2.append(i[:,40*(j):40*(j+1)])
    testu_labels2.append(testu_labels[labl])

In [ ]:
testu_images = np.array(testu_images2)
testu_images = testu_images.reshape(testu_images.shape+(1,))

testu_labels = np.array(testu_labels2)
testu_labels = testu_labels.astype(int)
testu_labels = to_categorical(testu_labels)

In [ ]:
testu_images.shape
#testu_labels.shape

(2794, 513, 40, 1)

In [ ]:
# To avoid repeated processing and trying models quickly save above arrays

with open('/content/gdrive/MyDrive/EE698V/train_label/train_label_task1_40.npy','wb') as f:
  np.save(f,train_labels)
with open('/content/gdrive/MyDrive/EE698V/train_label/train_images_task1_40.npy','wb') as f:
  np.save(f,train_images)
with open('/content/gdrive/MyDrive/EE698V/train_label/testu_images_task1_40.npy','wb') as f:
  np.save(f,testu_images)
with open('/content/gdrive/MyDrive/EE698V/train_label/testu_labels_task1_40.npy','wb') as f:
  np.save(f,testu_labels)

In [4]:
# Load directly and start training

with open('/content/gdrive/MyDrive/EE698V/train_label/train_label_task1_40.npy','rb') as f:
  train_labels = np.load(f)
with open('/content/gdrive/MyDrive/EE698V/train_label/train_images_task1_40.npy','rb') as f:
  train_images = np.load(f)
with open('/content/gdrive/MyDrive/EE698V/train_label/testu_images_task1_40.npy','rb') as f:
  testu_images = np.load(f)
with open('/content/gdrive/MyDrive/EE698V/train_label/testu_labels_task1_40.npy','rb') as f:
  testu_labels = np.load(f)

In [ ]:
print(train_labels.shape)
print(train_images.shape)
print(testu_images.shape)
print(testu_labels.shape)

In [15]:
# Final Model

model2 = Sequential()

model2.add(Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(513, 40, 1)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(4,1)))

model2.add(Conv2D(16,kernel_size=(3,3),activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(4,2)))

model2.add(Conv2D(16,kernel_size=(3,3),activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(4,2)))

model2.add(Dropout(0.8))
model2.add(Flatten())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.5))
# model2.add(Dense(64,activation='relu'))
# model2.add(Dropout(0.5))
model2.add(Dense(10,activation='softmax'))

model2.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 511, 38, 16)       160       
_________________________________________________________________
batch_normalization_9 (Batch (None, 511, 38, 16)       64        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 127, 38, 16)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 125, 36, 16)       2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 125, 36, 16)       64        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 18, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 16, 16)       

In [ ]:
# Fit the model 65 epochs
model2.fit(train_images, train_labels,validation_data=(testu_images,testu_labels), batch_size=4, epochs=65, verbose=1)

In [23]:
# Save the model
model2.save('/content/gdrive/MyDrive/EE698V/models/model_task1_40_65epochs_52')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/EE698V/models/model_task1_40_65+nepochs_57/assets


In [ ]:
# Load the model
model2 = tf.keras.models.load_model('/content/gdrive/MyDrive/EE698V/models/model_task1_40_65epochs_52')

In [6]:
from scipy import stats

In [ ]:
labl = ['air_conditioner','car_horn','children_playing','dog_bark','drilling','engine_idling','gun_shot','jackhammer','siren','street_music']

In [ ]:
# Load the task 1 test files, process them, generate outlabels, create csv file
final_test_path = '/content/gdrive/MyDrive/EE698V/test_task1/feats/'
save_path = '/content/gdrive/MyDrive/EE698V/test_task1/'
pred_file_name = []
pred_labels = []
for i in os.listdir(final_test_path):
  pred_file_name.append(i)
  test_image = np.load(final_test_path+i)
  iter = int(test_image.shape[1]/40)
  empty = []
  for j in range(iter):
    test_file = test_image[:,40*(j):40*(j+1)]
    test_file = test_file.reshape((1,)+test_file.shape + (1,))
    empty.append(np.argmax(model2.predict(test_file)))
  empty = np.array(empty)
  # print(empty,stats.mode(empty),sep='\t')
  pred_labels.append(labl[stats.mode(empty)[0][0]])
df = pd.DataFrame()
df['slice_file_name'] = pred_file_name
df['class'] = pred_labels
df.to_csv(save_path + 'task1_labels_test.csv')